## 공용 코드

In [1]:
# 파이썬
# ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 공통 모듈 임포트
import numpy as np
import pandas as pd
import os

# 깔끔한 그래프 출력을 위해 %matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)


# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "classification"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
# 이미지를 저장할 디렉토리 생성
os.makedirs(IMAGES_PATH, exist_ok=True)

# 이미지 저장
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장:", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

from matplotlib import font_manager, rc
import platform

path = "c:/Windows/Fonts/malgun.ttf"
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
    
    
mpl.rcParams['axes.unicode_minus'] = False
# Jupyter Notebook의 출력을 소수점 이하 3자리로 제한
%precision 3

# 그래픽 출력을 좀 더 고급화하기 위한 라이브러리
import seaborn as sns

# 과학 기술 통계 라이브러리
import scipy as sp
from scipy import stats

# 사이킷런 ≥0.20 필수
# 0.20 이상 버전에서 데이터 변환을 위한 Transformer 클래스가 추가됨
import sklearn
assert sklearn.__version__ >= "0.20"

# 노트북 실행 결과를 동일하게 유지하기 위해 시드 고정
# 데이터를 분할할 때 동일한 분할을 만들어 냄
np.random.seed(21)

# 자연어 처리

### 뉴스 그룹 분류 ( 이어서 )

In [2]:
# 데이터 가져오기
from sklearn.datasets import fetch_20newsgroups

news_data = fetch_20newsgroups(subset = 'all', random_state = 21)
# 가져온 데이터의 키 확인
# sklearn 에서 datasets 서브 패키지를 이용하면 가져온 데이터는 dict 형태
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [3]:
#print(news_data.filenames)
print(news_data.target_names)
# target 의 클래스 이름
#print(news_data.target)
# 데이터에 대한 설명

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [4]:
# target 의 분포 확인
# target 은 숫자로 되어 있음
# 분포가 한 쪽으로 치우치게 되면 데이터를 층화 추출 하거나 오버 샘플링,
# 언더 샘플링, 로그 변환 등을 고려해봐야 하므로 분포를 확인해봐야 함
# 정규 분포에서 성능이 가장 좋게 나옴
print(pd.Series(news_data.target).value_counts().sort_index())

0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64


In [5]:
# 데이터 가져오기

# header 나 footers, quoies 를 제거하고 가져오기
train_news = fetch_20newsgroups(subset = 'train', random_state = 21,
                               remove = ('headers', 'footers', 'quotes'))
# 훈련 데이터 생성
X_train = train_news.data
y_train = train_news.target # 실제 데이터에서 이게 없으면 군집
print(type(X_train)) # 문자열의 list
print(type(y_train)) # ndarray
#print(X_train[:5])

# 테스트 데이터
test_news = fetch_20newsgroups(subset = 'test', random_state = 21,
                               remove = ('headers', 'footers', 'quotes'))
X_test = test_news.data
y_test = test_news.target

<class 'list'>
<class 'numpy.ndarray'>


### 피처 벡터화

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

# 단어가 등장한 갯수 기반의 벡터화를 위한 인스턴스 생성
cnt_vect = CountVectorizer()

# 벡터화
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)
print(type(X_train_cnt_vect)) # sparse matrix 형태
print(type(X_train_cnt_vect[0])) 
print(type(X_train_cnt_vect[0][0][0])) 

<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>


### 로지스틱 회귀를 이용한 분류

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X_test_cnt_vect = cnt_vect.transform(X_test)

lr_clf = LogisticRegression(solver = 'lbfgs', max_iter = 1000)
lr_clf.fit(X_train_cnt_vect, y_train)
pred = lr_clf.predict(X_test_cnt_vect)

print('정확도 :', accuracy_score(y_test, pred))

정확도 : 0.5967870419543282


### TF-IDF 를 이용

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 벡터화에 TF-IDF 를 이용
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

# 벡터화 이후에는 동일하게 선형 회귀 적용
lr_clf = LogisticRegression(solver = 'lbfgs', max_iter = 1000)
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)

print('정확도 :', accuracy_score(y_test, pred))

정확도 : 0.6736590546999469


In [10]:
# TF-IDF 를 이용할 때 파라미터를 설정

# 위와 동일한 TF-IDF 방식에서 인스턴스를 생성할 때 stop words 를 추가
tfidf_vect = TfidfVectorizer(stop_words = 'english', 
                             ngram_range = (1, 2), max_df = 300)
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression(solver = 'lbfgs', max_iter = 1000)
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)

print('정확도 :', accuracy_score(y_test, pred))

정확도 : 0.6922464152947424


## 감성 분석

### 나이브 베이즈를 이용한 감성 분석

In [11]:
# 샘플 데이터
train = [('like you', 'pos'), ('do not like you', 'neg'),
        ('hate you', 'neg'), ('do not hate you', 'pos'),
        ('love you', 'pos'), ('do not love you', 'neg') ]
# TF-IDF 방식은 여러 문장에서 자주 등장하는 단어에 페널티를 부여해서
# 기여도를 낮추는 방식


# 등장한 모든 단어 찾기

from nltk.tokenize import word_tokenize # 단어별로 나누기
import nltk

#단어 단위로 분할해서 등장한 모든 단어 찾기
# 2중 for문을 사용하는데 더 큰 단위(sentence)가 앞에 옴
allwords = set(word.lower() for sentence in train
              for word in word_tokenize(sentence[0]))

print(allwords)



{'do', 'not', 'like', 'love', 'you', 'hate'}


In [12]:
# 분류기 만들기

# 단어 토큰화
t = [({word : (word in word_tokenize(x[0])) for word in allwords},
    x[1]) for x in train]
# 각 문장별로 단어를 포함하고 있는지 여부를 만들고
# 그 때의 감성(pos, neg)을 기록

print(t)

[({'do': False, 'not': False, 'like': True, 'love': False, 'you': True, 'hate': False}, 'pos'), ({'do': True, 'not': True, 'like': True, 'love': False, 'you': True, 'hate': False}, 'neg'), ({'do': False, 'not': False, 'like': False, 'love': False, 'you': True, 'hate': True}, 'neg'), ({'do': True, 'not': True, 'like': False, 'love': False, 'you': True, 'hate': True}, 'pos'), ({'do': False, 'not': False, 'like': False, 'love': True, 'you': True, 'hate': False}, 'pos'), ({'do': True, 'not': True, 'like': False, 'love': True, 'you': True, 'hate': False}, 'neg')]


In [13]:
# 텍스트 분류를 위한 나이브 베이즈 분류기를 이용해서 모델 생성
classifier = nltk.NaiveBayesClassifier.train(t)
classifier.show_most_informative_features()
# 각 feature(단어)에 대해 각 감성을 의미하는 비율을 학습해서 보여줌
# like, hate 와 같이 1.0:1.0 비율로 나타나는 feature는 불필요

Most Informative Features
                      do = False             pos : neg    =      1.7 : 1.0
                      do = True              neg : pos    =      1.7 : 1.0
                     not = False             pos : neg    =      1.7 : 1.0
                     not = True              neg : pos    =      1.7 : 1.0
                    hate = False             neg : pos    =      1.0 : 1.0
                    hate = True              neg : pos    =      1.0 : 1.0
                    like = False             neg : pos    =      1.0 : 1.0
                    like = True              neg : pos    =      1.0 : 1.0
                    love = False             neg : pos    =      1.0 : 1.0
                    love = True              neg : pos    =      1.0 : 1.0


In [14]:
# 예측

test_sentence = 'do not like flower'
test_sentence_features = {word.lower() : (word in word_tokenize(test_sentence.lower()))
                         for word in allwords}
print(test_sentence_features)
print(classifier.classify(test_sentence_features))
# 예측 결과는 neg

{'do': True, 'not': True, 'like': True, 'love': False, 'you': False, 'hate': False}
neg


### 한글 감성 분석

In [15]:
# 샘플 데이터
train = [('날씨가 좋다', '긍정'), ('날씨가 나쁘다', '부정'),
     ('날씨가 매우 좋다', '긍정'), ('날씨가 매우 나쁘다', '부정'),
     ('날씨가 맑다', '긍정'), ('비가 온다', '부정'),
        ('날이 시원하다', '긍정'), ('날이 너무 덥다', '부정')]

# 한글만 있다면 lower 함수는 필요 없음
allwords = set(word.lower() for sentence in train
              for word in word_tokenize(sentence[0]))

#print(allwords)

# 단어 토큰화
t = [({word : (word in word_tokenize(x[0])) for word in allwords},
    x[1]) for x in train]
#print(t)

# 모델 생성
classifier = nltk.NaiveBayesClassifier.train(t)
classifier.show_most_informative_features()

# 예측
test_sentence = '비가 오는데 날이 너무 덥다'
test_sentence_features = {word.lower() : (word in word_tokenize(test_sentence.lower()))
                         for word in allwords}
print(test_sentence_features)
# 조사 등의 문제로 인해 서로 다른 단어로 인식
# 형태소 분석이 필요함
print(classifier.classify(test_sentence_features))

Most Informative Features
                     나쁘다 = False              긍정 : 부정     =      1.8 : 1.0
                      좋다 = False              부정 : 긍정     =      1.8 : 1.0
                     날씨가 = False              부정 : 긍정     =      1.7 : 1.0
                     날씨가 = True               긍정 : 부정     =      1.4 : 1.0
                      너무 = False              긍정 : 부정     =      1.3 : 1.0
                      덥다 = False              긍정 : 부정     =      1.3 : 1.0
                      맑다 = False              부정 : 긍정     =      1.3 : 1.0
                      비가 = False              긍정 : 부정     =      1.3 : 1.0
                    시원하다 = False              부정 : 긍정     =      1.3 : 1.0
                      온다 = False              긍정 : 부정     =      1.3 : 1.0
{'시원하다': False, '매우': False, '나쁘다': False, '맑다': False, '좋다': False, '온다': False, '덥다': True, '날씨가': False, '너무': True, '날이': True, '비가': True}
부정


In [16]:
# 한글 형태소 분석
from konlpy.tag import Twitter

twitter = Twitter()

# 문장 단위로 형태소 분석기를 적용해서 
# 단어와 품사를 / 로 구분하여 추출
def tokenizing(doc):
    return ["/".join(t) for t in twitter.pos(doc, norm = True, stem = True)]

train_docs = [(tokenizing(row[0]), row[1]) for row in train]
print(train_docs)

C:\Users\USER\anaconda3\lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


[(['날씨/Noun', '가/Josa', '좋다/Adjective'], '긍정'), (['날씨/Noun', '가/Josa', '나쁘다/Adjective'], '부정'), (['날씨/Noun', '가/Josa', '매우/Noun', '좋다/Adjective'], '긍정'), (['날씨/Noun', '가/Josa', '매우/Noun', '나쁘다/Adjective'], '부정'), (['날씨/Noun', '가/Josa', '맑다/Adjective'], '긍정'), (['비/Noun', '가/Josa', '오다/Verb'], '부정'), (['날/Noun', '이/Josa', '시원하다/Adjective'], '긍정'), (['날/Noun', '이/Josa', '너무/Adverb', '덥다/Adjective'], '부정')]


In [17]:
# 감성을 빼고 단어만 추출하기
tokens = [t for d in train_docs for t in d[0]]
print(tokens)

['날씨/Noun', '가/Josa', '좋다/Adjective', '날씨/Noun', '가/Josa', '나쁘다/Adjective', '날씨/Noun', '가/Josa', '매우/Noun', '좋다/Adjective', '날씨/Noun', '가/Josa', '매우/Noun', '나쁘다/Adjective', '날씨/Noun', '가/Josa', '맑다/Adjective', '비/Noun', '가/Josa', '오다/Verb', '날/Noun', '이/Josa', '시원하다/Adjective', '날/Noun', '이/Josa', '너무/Adverb', '덥다/Adjective']


In [18]:
# 분류기 만들기 - 각 문장에 단어의 존재 여부를 확인해주는 함수
# 원리는 영어와 동일하나 한글이라 조금 더 복잡함
def term_exists(doc):
    return {word : (word in set(doc)) for word in tokens}

# 모든 문장을 해석해서 단어의 존재 여부와 감성을 가진 튜플의 리스트를 생성
train_xy = [(term_exists(d), c) for d, c in train_docs]
print(train_xy)
    

[({'날씨/Noun': True, '가/Josa': True, '좋다/Adjective': True, '나쁘다/Adjective': False, '매우/Noun': False, '맑다/Adjective': False, '비/Noun': False, '오다/Verb': False, '날/Noun': False, '이/Josa': False, '시원하다/Adjective': False, '너무/Adverb': False, '덥다/Adjective': False}, '긍정'), ({'날씨/Noun': True, '가/Josa': True, '좋다/Adjective': False, '나쁘다/Adjective': True, '매우/Noun': False, '맑다/Adjective': False, '비/Noun': False, '오다/Verb': False, '날/Noun': False, '이/Josa': False, '시원하다/Adjective': False, '너무/Adverb': False, '덥다/Adjective': False}, '부정'), ({'날씨/Noun': True, '가/Josa': True, '좋다/Adjective': True, '나쁘다/Adjective': False, '매우/Noun': True, '맑다/Adjective': False, '비/Noun': False, '오다/Verb': False, '날/Noun': False, '이/Josa': False, '시원하다/Adjective': False, '너무/Adverb': False, '덥다/Adjective': False}, '긍정'), ({'날씨/Noun': True, '가/Josa': True, '좋다/Adjective': False, '나쁘다/Adjective': True, '매우/Noun': True, '맑다/Adjective': False, '비/Noun': False, '오다/Verb': False, '날/Noun': False, '이/Josa': False, '시원하다/Adj

In [19]:
classifier = nltk.NaiveBayesClassifier.train(train_xy)
classifier.show_most_informative_features()

Most Informative Features
           나쁘다/Adjective = False              긍정 : 부정     =      1.8 : 1.0
            좋다/Adjective = False              부정 : 긍정     =      1.8 : 1.0
                 날씨/Noun = False              부정 : 긍정     =      1.7 : 1.0
                 날씨/Noun = True               긍정 : 부정     =      1.4 : 1.0
               너무/Adverb = False              긍정 : 부정     =      1.3 : 1.0
            덥다/Adjective = False              긍정 : 부정     =      1.3 : 1.0
            맑다/Adjective = False              부정 : 긍정     =      1.3 : 1.0
                  비/Noun = False              긍정 : 부정     =      1.3 : 1.0
          시원하다/Adjective = False              부정 : 긍정     =      1.3 : 1.0
                 오다/Verb = False              긍정 : 부정     =      1.3 : 1.0


In [20]:
test_sentence = [('날씨를 보니 곧 비가 온 뒤에 더워질 예정이래')]
test_docs = twitter.pos(test_sentence[0])
test_sentence_features = {word : (word in tokens) for word in test_docs}
print(classifier.classify(test_sentence_features))

부정


### IMDB 데이터를 이용한 감성 분석

In [21]:
# 데이터 가져오기
# tsv 파일이므로 seperator 는 탭
review_df = pd.read_csv('./data/kaggle_movie/labeledTrainData.tsv', header = 0,
                       sep = '\t', quoting = 3)
# id 는 review 를 구분하기 위한 데이터
# sentiment 는 감성인데 1이면 긍정이고 2면 부정
# review 는 리뷰 데이터
print(review_df.head())

         id  sentiment                                             review
0  "5814_8"          1  "With all this stuff going down at the moment ...
1  "2381_9"          1  "\"The Classic War of the Worlds\" by Timothy ...
2  "7759_3"          0  "The film starts with a manager (Nicholas Bell...
3  "3630_4"          0  "It must be assumed that those who praised thi...
4  "9495_8"          1  "Superbly trashy and wondrously unpretentious ...


In [22]:
# 정규식을 이용해서 불필요한 데이터 제거
import re

# 줄바꿈을 공백으로 제거
review_df['review'] = review_df['review'].str.replace('<br/ >', ' ')
# 영어가 아닌 것들은 공백으로 그 이외의 것들은 그대로 사용
review_df['review'] = review_df['review'].apply(lambda x : re.sub('[^a-zA-Z]', ' ', x))

# 확인 - 따옴표, \ 등 불필요한 내용이 제거됨
print(review_df['review'].head())

0     With all this stuff going down at the moment ...
1       The Classic War of the Worlds   by Timothy ...
2     The film starts with a manager  Nicholas Bell...
3     It must be assumed that those who praised thi...
4     Superbly trashy and wondrously unpretentious ...
Name: review, dtype: object


In [23]:
# 훈련 데이터와 테스트 데이터를 분할
from sklearn.model_selection import train_test_split

# 타겟
class_df = review_df['sentiment']
# 피처
feature_df = review_df.drop(['id', 'sentiment'], axis = 1, inplace = False)

print(class_df.head())
print(feature_df.head())

X_train, X_test, y_train, y_test = train_test_split(feature_df, class_df,
                                                   test_size = 0.3, random_state = 21)

print(X_train.shape, X_test.shape)

0    1
1    1
2    0
3    0
4    1
Name: sentiment, dtype: int64
                                              review
0   With all this stuff going down at the moment ...
1     The Classic War of the Worlds   by Timothy ...
2   The film starts with a manager  Nicholas Bell...
3   It must be assumed that those who praised thi...
4   Superbly trashy and wondrously unpretentious ...
(17500, 1) (7500, 1)


In [24]:
# 훈련 및 예측
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

# 파이프라인 생성
# CountVectorizer 를 사용
# TF-IDF 를 사용할 수도 있음
pipeline = Pipeline([
    ('cnt_vect', CountVectorizer(stop_words = 'english', ngram_range = (1, 2))),
    ('lr_clf', LogisticRegression(C = 10))
])

pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:, 1]

print('정확도 :', accuracy_score(y_test, pred))
print('ROC_AUC :', roc_auc_score(y_test, pred_probs))

C:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


정확도 : 0.8804
ROC_AUC : 0.9484153149245135


In [25]:
# 파이프라인 생성 - TF-IDF 적용
pipeline = Pipeline([
    ('cnt_vect', TfidfVectorizer(stop_words = 'english', ngram_range = (1, 2))),
    ('lr_clf', LogisticRegression(C = 10))
])

pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:, 1]

print('정확도 :', accuracy_score(y_test, pred))
print('ROC_AUC :', roc_auc_score(y_test, pred_probs))
# 영문의 경우 TF-IDF 를 적용했을 때 좀 더 나은 결과를 보임

C:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


정확도 : 0.8893333333333333
ROC_AUC : 0.958380967280847


## 네이버 식당 리뷰 데이터를 이용한 한글 지도 학습 기반의 감성 분석

### 데이터 가져오기 

In [26]:
df = pd.read_csv('./data/review_data.csv')
print(df.head())

   score                      review  y
0      5            친절하시고 깔끔하고 좋았습니다  1
1      5                  조용하고 고기도 굿  1
2      4      갈비탕과 냉면, 육회비빔밥이 맛있습니다.  1
3      4  대체적으로 만족하나\n와인의 구성이 살짝 아쉬움  1
4      5       고기도 맛있고 서비스는 더 최고입니다~  1


In [27]:
# 한글 데이터만 추출하는 함수
def korean_text(text):
    # ㄱ-ㅣ 는 ㅋ 과 같은 초성을 추출
    # 일반 한글은 가-힣 으로 추출
    hangle = re.compile('[^ ㄱ-ㅣ 가-힣]')
    result = hangle.sub('',text)
    
    return result

# 온점, 물결표 등을 삭제하고 한글만 남김
df['ko_text'] = df['review'].apply(lambda x : korean_text(x))
del df['review']

print(df.head())

   score  y                   ko_text
0      5  1          친절하시고 깔끔하고 좋았습니다
1      5  1                조용하고 고기도 굿
2      4  1      갈비탕과 냉면 육회비빔밥이 맛있습니다
3      4  1  대체적으로 만족하나와인의 구성이 살짝 아쉬움
4      5  1      고기도 맛있고 서비스는 더 최고입니다


### 형태소 분석

In [28]:
from konlpy.tag import Okt

# 단어와 품사의 조합으로 변환
def get_pos(x):
    tagger = Okt()
    pos = tagger.pos(x)
    pos = ['{}/{}'.format(word, tag) for word, tag in pos]
    
    return pos

# 하나의 데이터로 확인하기
result = get_pos(df['ko_text'][1])
print(result)

['조용하고/Adjective', '고기/Noun', '도/Josa', '굿/Noun']


### 피처 벡터화

In [29]:
# TF-IDF 방식을 사용해서 벡터화

index_vectorizer = TfidfVectorizer(tokenizer = lambda x : get_pos(x))
X = index_vectorizer.fit_transform(df['ko_text'].tolist())
print(X.shape)
print(type(X)) # sparse matrix 클래스

C:\Users\USER\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


(545, 3030)
<class 'scipy.sparse._csr.csr_matrix'>


In [30]:
# 피처 확인
print(str(index_vectorizer.vocabulary_)[:100])
print(df['ko_text'][0])
print(X[0])
# sparse matrix 형태로 저장
# TF-IDF 를 적용했기 때문에 sparse 와 가중치가 출력됨

{'친절하시고/Adjective': 2647, '깔끔하고/Adjective': 428, '좋았습니다/Adjective': 2403, '조용하고/Adjective': 2356, '고
친절하시고 깔끔하고 좋았습니다
  (0, 2403)	0.48955631270748484
  (0, 428)	0.6726462183300624
  (0, 2647)	0.5548708693511647


In [31]:
# 학습 데이터와 훈련 데이터 생성

y = df['y']
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                            test_size = 0.3, random_state = 21)
print(X_train.shape)
print(X_test.shape)

(381, 3030)
(164, 3030)


In [32]:
# 모델 훈련 및 평가 - 선형 회귀 사용

from sklearn.metrics import precision_score, recall_score, f1_score

lr = LogisticRegression(random_state = 21)
lr.fit(X_train, y_train)
pred = lr.predict(X_test)
pred_probs = lr.predict_proba(X_test)[:, 1]

print('정확도 :', accuracy_score(y_test, pred))
print('정밀도 :', precision_score(y_test, pred))
print('재현도 :', recall_score(y_test, pred))
print('f1 score :', f1_score(y_test, pred))
print('ROC_AUC :', roc_auc_score(y_test, pred_probs))

정확도 : 0.9024390243902439
정밀도 : 0.9024390243902439
재현도 : 1.0
f1 score : 0.9487179487179488
ROC_AUC : 0.9303209459459459


In [33]:
# 오차 행렬 확인

from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y_test, pred)
print(conf_matrix)
# 모든 결과를 1로 예측한걸 확인 가능

[[  0  16]
 [  0 148]]


In [34]:
# 타겟의 분포 확인

df['y'].value_counts()
# 1을 나타내는 데이터가 0의 10배

1    492
0     53
Name: y, dtype: int64

In [35]:
# Undersampling 적용

# 데이터를 50개만 임의로 선정해서 추출
positive_random_idx = df[df['y'] == 1].sample(50, random_state = 21).index.tolist()
negative_random_idx = df[df['y'] == 0].sample(50, random_state = 21).index.tolist()

# 임의로 추출한 인덱스의 데이터만 가져옴
random_idx = positive_random_idx + negative_random_idx
sample_X = X[random_idx, :]
y = df['y'][random_idx]

X_train, X_test, y_train, y_test = train_test_split(sample_X, y,
                                            test_size = 0.25, random_state = 21)

# 모델에 적용
lr = LogisticRegression(random_state = 21)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
y_pred_probs = lr.predict_proba(X_test)[:, 1]

print('정확도 :', accuracy_score(y_test, y_pred))
print('정밀도 :', precision_score(y_test, y_pred))
print('재현도 :', recall_score(y_test, y_pred))
print('f1 score :', f1_score(y_test, y_pred))
print('ROC_AUC :', roc_auc_score(y_test, y_pred_probs))

정확도 : 0.44
정밀도 : 1.0
재현도 : 0.17647058823529413
f1 score : 0.3
ROC_AUC : 0.9044117647058824


In [36]:
# 오차 행렬 확인
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)
# 이전과 다르게 0으로 예측하는 경우와 1로 예측하는 경우 모두 생김
# 데이터가 부족해서 정확도는 낮게 나오는듯 함

[[ 8  0]
 [14  3]]


## 토픽 모델링

In [37]:
# 데이터 가져오기

from sklearn.decomposition import LatentDirichletAllocation
# 차원 축소에 관련된 일을 하는 decomposition 패키지

# 데이터를 가져올 카테고리를 설정
categories = ['rec.motorcycles', 'rec.sport.baseball', 'comp.graphics', 'comp.windows.x',
'talk.politics.mideast', 'soc.religion.christian', 'sci.electronics', 'sci.med' ]
# sci 는 과학, soc 는 사회, rec 는 레크레이션 등

# 카테고리에 해당하는 데이터만 가져오기
news_data = fetch_20newsgroups(subset = 'all', random_state = 21,
                              remove = ('headers', 'footers', 'quotes'),
                              categories = categories)

In [38]:
# vectorize
count_vect = CountVectorizer(max_df = 0.95, max_features = 1000, min_df = 2,
                            stop_words = 'english', ngram_range = (1, 2))

feat_vect = count_vect.fit_transform(news_data.data)
print('Count Vectorizer Shape :', feat_vect.shape)
#  (7862, 1000)

Count Vectorizer Shape : (7862, 1000)


In [39]:
# 토픽 모델링

# 8개로 숫자를 지정해서 토픽을 나눔 - 군집과 유사
lda = LatentDirichletAllocation(n_components = 8, random_state = 21)
lda.fit(feat_vect)
print(lda.components_.shape)
# (8, 1000)
# 7862였던 데이터를 8개 토픽으로 분류

(8, 1000)


In [40]:
# 각 토픽에서 중요 N개 단어 추출하는 함수
def display_topics(model, feature_names, num_top_words):
    for topic_index, topic in enumerate(model.components_):
        print('Topic #', topic_index + 1, sep = '')
    
        topic_word_indexes = topic.argsort()[::-1]
        # 지정한 갯수에 해당하는 단어들의 인덱스를 가져옴
        top_indexes = topic_word_indexes[:num_top_words]

        # 인덱스를 사용해서 지정한 갯수의 단어를 조합
        feature_concat = ' '.join([feature_names[i] for i in top_indexes])

        print(feature_concat)

# 피처 이름 가져오기
# sklearn 의 최신 버전에서 get_feature_names 함수가 사라지고
# get_feature_names_out 함수로 바뀜
feature_names = count_vect.get_feature_names_out()


display_topics(lda, feature_names, 15)

Topic #1
don just like know think time people said year good didn say did going ll
Topic #2
10 medical health 1993 12 disease research cancer patients 20 92 11 april information number
Topic #3
god people jesus church believe think christ does say christian christians know bible don faith
Topic #4
file image jpeg program color gif output files format images entry bit use display 24
Topic #5
like use know does just thanks don problem used ve help need good work want
Topic #6
israel jews jewish israeli people arab state edu arabs ed war world right peace country
Topic #7
edu graphics available window com server ftp windows software dos mail version data sun motif
Topic #8
armenian armenians turkish people dos dos turkey armenia 000 genocide government turks russian azerbaijan greek said


## 텍스트 군집

In [48]:
# data/OpinosisDataset/topics 디렉토리에서 데이터 파일 전부 읽기

import glob, os
import platform

# 디렉토리의 이름(경로)을 생성
# MAC 은 경로에 / 를 사용하고 Windows 는 \\(\) 를 사용
# 파이썬은 제어문이 블럭이 아니라서 제어문 안에서 변수를 생성해도 되지만
# 자바와 같은 다른 언어는 제어문이 블럭이므로 변수 이름을 미리 만들고 사용해야 함
path_name = ""
if platform.system() == 'Darwin':
    path_name = './data/OpinosisDataset/topics'
elif platform.system() == 'Windows':
    path_name = '.\\data\\OpinosisDataset\\topics'

# 디렉토리 안의 모든 파일 이름을 list 로 생성
# path_name 내의'.data' 로 끝나는 모든 파일 이름(파일의 경로)을 가져옴
# ~로 시작하는, ~를 중간에 포함하는 등의 방식도 지정 가능
all_file_name = glob.glob(os.path.join(path_name, "*.data"))
print(all_file_name[:5])

['.\\data\\OpinosisDataset\\topics\\accuracy_garmin_nuvi_255W_gps.txt.data', '.\\data\\OpinosisDataset\\topics\\bathroom_bestwestern_hotel_sfo.txt.data', '.\\data\\OpinosisDataset\\topics\\battery-life_amazon_kindle.txt.data', '.\\data\\OpinosisDataset\\topics\\battery-life_ipod_nano_8gb.txt.data', '.\\data\\OpinosisDataset\\topics\\battery-life_netbook_1005ha.txt.data']


In [50]:
# 파일의 이름을 저장할 list 
file_name_list = []
# 파일의 내용을 저장할 list
file_in = []

# 파일의 경로를 순회하면서 파일의 내용을 읽어서 하나로 만들기

# 파일의 내용 읽기
for file in all_file_name:
    df = pd.read_table(file_name, index_col = None, header = 0, encoding = 'latin')